In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir('../')

In [12]:
%pwd

'd:\\Programming\\Ineuron\\ML-Text-summarization'

- Update config.yaml first

### Update Entity

In [13]:
from dataclasses import dataclass
from pathlib import Path

# @dataclass(frozen=True)
# class DataTransformationConfig:
#     root_dir: Path
#     data_path: Path
#     tokenizer_name: Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

Update configuration manager

In [14]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [15]:
# class ConfigurationManager:
#     def __init__(
#             self,
#             config_filepath=CONFIG_FILE_PATH,
#             params_filepath=PARAMS_FILE_PATH):
        
#         self.config = read_yaml(config_filepath)
#         self.params = read_yaml(params_filepath)

#         create_directories([self.config.artifacts_root])

#     def get_data_transformation_config(self)-> DataTransformationConfig:
#         config = self.config.data_transformation

#         create_directories([config.root_dir])

#         data_transformation_config = DataTransformationConfig(
#             root_dir=config.root_dir,
#             data_path=config.data_path,
#             tokenizer_name=config.tokenizer_name,
#         )

#         return data_transformation_config
    


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config



Create Components

In [16]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [17]:
# class DataTransformation:
#     def __init__(self, config:DataTransformationConfig):
#         self.config = config
#         self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

#     def convert_example_to_features(self,example_batch):
#         input_encoding = self.tokenizer(example_batch['dialogue'],max_length = 1024, truncation= True)

#         with self.tokenizer.as_target_tokenizer():
#             target_encoding = self.tokenizer(example_batch['summary'],max_length = 128, truncation= True)

#         return {
#             'input_ids':input_encoding['input_ids'],
#             'attention_mask':input_encoding['attention_mask'],
#             'labels':target_encoding['input_ids']
#         }

#     def covert(self):
#         dataset_samsum = load_from_disk(self.config.data_path)
#         dataset_samsum_pt = dataset_samsum.map(self.convert_example_to_features,batched=True)
#         dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        print("data load")
        dataset_samsum = load_from_disk(self.config.data_path)
        print("data load done and mapping started")
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        print("data mapping done and dataset save")
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

Pipeline

In [21]:
# try:
#     config = ConfigurationManager()
#     data_transformation_config = config.get_data_transformation_config()
#     data_transformation = DataTransformation(config=data_transformation_config)
#     data_transformation.convert()
# except Exception as e:
#     raise e



try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e



[2023-08-23 10:19:13,622: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-23 10:19:13,624: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-23 10:19:13,625: INFO: common: created directory at: artifacts]
[2023-08-23 10:19:13,626: INFO: common: created directory at: artifacts/data_transformation]


TypeError: Couldn't build proto file into descriptor pool: duplicate file name sentencepiece_model.proto